# Refactoring Dataset Importing

## Note-centric dataset

Rather than viewing the data as a collection of data sources (such as MAPS), each containing a collection of instruments, each of which contains a collection of note samples, I would like to conside the notes themselves as the primary assets of the dataset.

I will define a dataset class, which contains a collection of notes.

Each note will contain a collection of samples from various instruments from various datasets.

Data source classes will be used to populate the dataset.

## Notes

A `Note` object represents a single note from C0 to B9.

Each note has a *note index*. The index for a note is a count of how many semitones that note is above C♭0 (that is, the B immediately below C0). This means that the note index for C0 is 1, while B9's index is 120.

Note index 0 is used to represent the lack of a note. This could mean silence, but in the context of pitch detection it could also mean sound (non-silence) that does not contain any discernable notes.

The `Note` object contains a list of `NoteSamples` for that note, drawn from a variety of data sources and instruments.

In [1]:
class Note:
    def __init__(self, index):
        self.index = index
        self.samples = []
        
    def add_sample(self, sample):
        self.samples.append(sample)


class Silence:
    pass

## Note samples

Each `NoteSample` provides access to the audio waveform of a single note. Details of how the waveform is stored and/or accessed from its data source are provided by specialisations of the `NoteSample` class.

Note samples can provide subsets of their waveform data as *clips*. A clip represents a portion of the waveform that could be returned from an audio API as one recording *window* (or *buffer*).

The *onset* of a note is the time at which that note's key was struck. The *onset clip* is clip of the desired length starting at the onset of the note.

The *offset* of a note is the time at which it's key was released. A *random clip* is clip of the desired length starting at, or after, the onset of the note - and ending at, or before, the note's offset.

Onset and offset "times" are represented as indexes into the note sample's waveform array.

In [2]:
class NoteSample:
    def __init(self, onset, offset):
        self.onset = onset
        self.offset = offset
    
    def get_onset_clip(self, window_size):
        assert offset > self.onset + window_size
        return self.get_audio()[self.onset:(self.onset + window_size)]
        
    def get_random_clip(self, window_size):
        pass



### File-based note samples

The waveform is loaded lazily, and is then cached. The cache can be cleared at any time.

In [3]:
class FileBasedNoteSample:
    def __init(self, filename):
        self.filename = filename
        self.waveform = None
    
    def clear():
        self.waveform = None
        
    def get_waveform():
        return self.waveform

## The dataset

The dataset reprsents the collection of all note samples, grouped by note, from all the instruments in all the datasources.

Typical usage would look something like:

``` python
    dataset = Dataset()
    dataset.add_datasource(MAPS('/datasets/audio/maps'))
    dataset.add_datasource(...)
    dataset.add_datasource(...)
    
    for note in dataset.notes:
        ...
```

In [4]:
class Dataset:
    MAX_NOTE_INDEX = 121
    
    def __init__(self):
        self.notes = [Silence()]
        self.notes.extend(Note(index) for index in range(1, Dataset.MAX_NOTE_INDEX + 1))
        pass
    
    def add_datasource(self, datasource):
        for sample in datasource.get_samples:
            self.notes[sample.note_index].add_sample(sample)

## Data sources

In [5]:
class DataSource:
    def get_samples():
        pass
    
    
class MAPS(DataSource):
    def __init__(self, root_directory):
        pass
    
    def get_samples():
        pass